# Learn Agent

Learns:
- https://codelabs.developers.google.com/codelabs/gemini-function-calling#3
- done: https://python.langchain.com/docs/modules/agents/how_to/custom_agent
- done: https://python.langchain.com/docs/modules/agents/agent_types/structured_chat

In [1]:
from langchain_community.tools.tavily_search import TavilySearchResults
import sys
sys.path.append("../../")
from src.config import Configuration

conf = Configuration()

/home/h4438/miniconda3/envs/uni/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Search tool

In [2]:
conf.set_tavily_token()
tavily = TavilySearchResults()

In [4]:
a = tavily.invoke(input="what is langchain")
a

[{'url': 'https://python.langchain.com/docs/get_started/introduction',
  'content': "Modules\u200b\nLangChain provides standard, extendable interfaces and integrations for the following modules:\nInterface with language models\nInterface with application-specific data\nLet models choose which tools to use given high-level directives\nExamples, ecosystem, and resources\u200b\nUse cases\u200b\nWalkthroughs and techniques for common end-to-end use cases, like:\nIntegrations\u200b\nLangChain is part of a rich ecosystem of tools that integrate with our framework and build on top of it. Together, these products simplify the entire application lifecycle:\nLangChain Libraries\u200b\nThe main value props of the LangChain packages are:\nOff-the-shelf chains make it easy to get started. API reference\u200b\nHead to the reference section for full documentation of all classes and methods in the LangChain and LangChain Experimental Python packages.\n Developer's guide\u200b\nCheck out the developer'

## Local search tool

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = FAISS.from_documents(documents, conf.get_gemini_embeddings())
retriever = vector.as_retriever()

In [4]:
a = retriever.get_relevant_documents("how to upload a dataset")[0]
a

Document(page_content='LangSmith Overview and User Guide | 🦜️🛠️ LangSmith', metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': 'LangSmith Overview and User Guide | 🦜️🛠️ LangSmith', 'description': 'Building reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work needed to bring the performance of prompts, chains and agents up the level where they are reliable enough to be used in production.', 'language': 'en'})

## Normal tool

In [4]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

from langchain.agents import tool


@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)


get_word_length.invoke("abc")

3

## Agent

In [7]:
from langchain import hub
from langchain.agents import AgentExecutor, create_structured_chat_agent
tools = [tavily, retriever_tool, get_word_length]
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/structured-chat-agent")

# Choose the LLM that will drive the agent
llm = conf.get_gemini_pro(True)

# Construct the JSON agent
agent = create_structured_chat_agent(llm, tools, prompt)

In [8]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

In [9]:
from langsmith.run_helpers import traceable

conf.enable_tracing(project="LEARN")

@traceable(tags=["agent"])
def run_agent(q:str):
    a = agent_executor.invoke({"input": q})
    return a

In [11]:
q = "how many characters in 'hello world'"
a = run_agent(q)
a



> Entering new AgentExecutor chain...
```
{
  "action": "get_word_length",
  "action_input": {
    "word": "hello world"
  }
}
```11```
{
  "action": "Final Answer",
  "action_input": "Eleven"
}
```

> Finished chain.


{'input': "how many characters in 'hello world'", 'output': 'Eleven'}